In [5]:
import re, os, string, warnings, json
warnings.filterwarnings("ignore")
import math
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt', quiet=True)
from collections import Counter
from rapidfuzz import fuzz
from word2number import w2n
import dateparser
from dateutil import parser as duparser

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("synthetic_dataset.csv", encoding_errors="replace")

In [3]:
# This is being done so as to confirm to a same schema for data throughout the project
# Heuristic rename if your headers differ slightly
rename_map = {
    "reference": "gold_reference",
    "gold": "gold_reference",
    "answer": "ai_response",
    "response": "ai_response",
}
for k,v in rename_map.items():
    if k in df.columns and v not in df.columns:
        df.rename(columns={k:v}, inplace=True)

# Ensure exactly the 4 we use exist
for c in ["question","context","gold_reference","ai_response"]:
    if c not in df.columns: df[c] = ""

def _clean(s): 
    return (str(s) if pd.notna(s) else "").strip()

UNANSWERABLE_MARKERS = {
    "unanswerable","not answerable","cannot be determined","insufficient context",
    "not available in passage","unknown","na","n/a"
}

def derive_flags(row):
    gold = _clean(row["gold_reference"]).lower()
    ctx  = _clean(row["context"])
    is_unanswerable = gold in UNANSWERABLE_MARKERS
    context_missing = (ctx == "")
    no_gold = (gold == "")  # empty gold means judge vs context only
    return pd.Series({
        "is_unanswerable": is_unanswerable,
        "context_missing": context_missing,
        "no_gold": no_gold
    })

flags = df.apply(derive_flags, axis=1)
df = pd.concat([df, flags], axis=1)
df.head(3)

question  \
0  What did the Court of Justice not acknowledge?   
1  What did the Court of Justice not acknowledge?   
2  What did the Court of Justice not acknowledge?   

                                             context gold_reference  \
0  Fourth, national courts have a duty to interpr...            NaN   
1  Fourth, national courts have a duty to interpr...            NaN   
2  Fourth, national courts have a duty to interpr...            NaN   

                                         ai_response  is_unanswerable  \
0                    The passage doesn’t state this.            False   
1  It states that what did the court of justice w...            False   
2  The decision was influenced by the Paris Conve...            False   

   context_missing  no_gold  
0            False     True  
1            False     True  
2            False     True

In [6]:
# ---------- custom functions for various use cases ----------
def to_text(x) -> str:
    if x is None:
        return ""
    try:
        import pandas as _pd
        if _pd.isna(x):
            return ""
    except Exception:
        pass
    return str(x)

def normalize_text(s: str) -> str:
    s = to_text(s).lower().strip()
    return re.sub(r"\s+", " ", s)

def safe_bool(x) -> bool:
    s = to_text(x).strip().lower()
    if s in {"1","true","yes","y","t"}: return True
    return False

def extract_numbers(s: str):
    return re.findall(r"\d+(?:\.\d+)?", to_text(s))

def extract_entities_simple(raw: str):
    raw = to_text(raw)
    # Heuristic: capitalized tokens + multi-word proper nouns
    multi = re.findall(r"\b([A-Z][a-z]+(?:\s+[A-Z][a-z]+)+)\b", raw)
    single = re.findall(r"\b[A-Z][a-z]+\b", raw)
    ents = set(single)
    for phrase in multi:
        ents.add(phrase)
        for part in phrase.split():
            ents.add(part)
    return list(ents)

def no_answer_template(s: str) -> bool:
    s = normalize_text(s)
    patterns = [
        "no answer","not in passage","not within passage","not stated",
        "cannot be determined","cannot be found","insufficient information",
        "unknown","not specified","no information","answer not available",
        "cannot determine","not provided","information is missing"
    ]
    return any(p in s for p in patterns)

In [ ]:
# Core hallucination scoring
# Hallucination = groundedness w.r.t. context.
# Here we are considering and scoring how well is the ai_response correlated with the context given by user
# if no context is there, score is 0.5 as the probability of ai_response to be hallucinating is assumed to be 50%
# designing is such that --> higher hallucination score == less hallucination 


def hallucination_score(row, tau_label=0.75, weights=(0.5,0.3,0.2)):
    """
    Returns dict with hallu_score, hallu_label, and sub-scores.
    """
    
    ctx_raw  = to_text(row.get("context",""))
    resp_raw = to_text(row.get("ai_response",""))

    # Flags
    is_unans        = safe_bool(row.get("is_unanswerable", False))
    context_missing = safe_bool(row.get("context_missing", False))

    # If no context → neutral
    if context_missing or not ctx_raw.strip():
        return {
            "hallu_score": 0.5,
            "hallu_label": int(0.5 >= tau_label),
            "sim": np.nan, "ent": np.nan, "claim": np.nan,
            "support_used": "none",
            "no_answer_template": no_answer_template(resp_raw)
        }

    # Normalized forms
    ctx_norm  = normalize_text(ctx_raw)
    resp_norm = normalize_text(resp_raw)

    # ---- TF-IDF cosine similarity ----
    sim = 0.0
    if ctx_norm.strip() and resp_norm.strip():
        vect = TfidfVectorizer(min_df=1, ngram_range=(1,2))
        try:
            tfidf = vect.fit_transform([ctx_norm, resp_norm])
            sim = float(cosine_similarity(tfidf[0], tfidf[1])[0,0])
        except ValueError:
            sim = 0.0

    # ---- Entity/number overlap ----
    resp_ents = set(extract_entities_simple(resp_raw))
    ctx_ents  = set(extract_entities_simple(ctx_raw))
    resp_nums = set(extract_numbers(resp_norm))
    ctx_nums  = set(extract_numbers(ctx_norm))
    total_claims = max(len(resp_ents) + len(resp_nums), 1)
    ent_hits = len(resp_ents & ctx_ents)
    num_hits = len(resp_nums & ctx_nums)
    ent_score = (ent_hits + num_hits) / total_claims

    # ---- Claim density (token overlap) ----
    resp_tokens = set(resp_norm.split())
    ctx_tokens  = set(ctx_norm.split())
    claim_score = len(resp_tokens & ctx_tokens) / (len(resp_tokens) or 1)

    # ---- Combine ----
    w_sim, w_ent, w_claim = weights
    final = w_sim*sim + w_ent*ent_score + w_claim*claim_score

    # ---- Unanswerable logic ----
    template = no_answer_template(resp_raw)
    if is_unans:
        if template and ent_score > 0.8:
            final = min(1.0, final+0.2)
        elif not template:
            final = min(final, 0.6)

    final = float(min(1.0, max(0.0, final)))
    label = int(final >= tau_label)

    return {
        "hallu_score": final,
        "hallu_label": label,
        "sim": sim,
        "ent": ent_score,
        "claim": claim_score,
        "support_used": "context",
        "no_answer_template": template
    }

def add_hallucination_scores(df, tau_label=0.75, weights=(0.5,0.3,0.2)):
    rows = [hallucination_score(r, tau_label, weights) for _, r in df.iterrows()]
    return pd.concat([df.reset_index(drop=True), pd.DataFrame(rows)], axis=1)

In [8]:
df = add_hallucination_scores(df, tau_label=0.75)
df[["ai_response","hallu_score","hallu_label","sim","ent","claim","support_used"]].head()

ai_response  hallu_score  \
0                    The passage doesn’t state this.     0.443529   
1  It states that what did the court of justice w...     0.176042   
2  The decision was influenced by the Paris Conve...     0.182145   
3                               No answer in passage     0.118184   
4  Bullet 1) The Court accepted horizontal effect...     0.245052   

   hallu_label       sim       ent     claim support_used  
0            0  0.127059  1.000000  0.400000      context  
1            0  0.177084  0.000000  0.437500      context  
2            0  0.173702  0.200000  0.176471      context  
3            0  0.036368  0.000000  0.500000      context  
4            0  0.107962  0.428571  0.312500      context

In [9]:
df.to_csv("scored datasets/hallucination_scores_added.csv")